Source: https://github.com/chrise96/image-to-coco-json-converter

In [1]:
!pip install Pillow==7.2.0
!pip install Shapely==1.7.1
!pip install scikit-image==0.17.2
!pip install numpy==1.18.5

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
/bin/bash: switchml: line 1: syntax erro

In [2]:
import glob
from create_annotations import *

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".jpg"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                                
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id

In [12]:
# Label ids of the dataset
category_ids = {
    "environment": 0,
    "rail": 1,
}

# Define which colors match which categories in the images
category_colors = {
    "(0, 0, 0)": 0, # Environment
    "(255, 255, 255)": 1, # Rail
}

# Define the ids that are a multiplolygon. In this case: rail
multipolygon_ids = [0,1]

# Get the standard COCO JSON format
coco_format = get_coco_json_format()
    
for keyword in ["train", "val"]:
    mask_path = "test_dataset/{}_mask/".format(keyword)
    print("Converting to COCO dataset... (dit duurt lang)")
        
    # Create category section
    coco_format["categories"] = create_category_annotation(category_ids)
    
    # Create images and annotations sections
    coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    with open("test_dataset_COCO/{}.json".format(keyword),"w") as outfile:
        print("done")
        json.dump(coco_format, outfile)
        
    print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

Converting to COCO dataset... (dit duurt lang)
done
Created 8 annotations for images in folder: test_dataset/train_mask/
Converting to COCO dataset... (dit duurt lang)
done
Created 4 annotations for images in folder: test_dataset/val_mask/
